# 1. Uczenie klasyfikatora

Trochę informacji: https://towardsdatascience.com/transfer-learning-from-pre-trained-models-f2393f124751

In [3]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, GlobalAveragePooling2D, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

In [4]:
trdata_gen = ImageDataGenerator()
traindata = trdata_gen.flow_from_directory(directory="../data/Training",target_size=(224,224)) 

Found 67692 images belonging to 131 classes.


In [5]:
CLASSES = len(traindata.class_indices)

In [6]:
tsdata_gen = ImageDataGenerator()
testdata = tsdata_gen.flow_from_directory(directory="../data/Test", target_size=(224,224))

Found 22688 images belonging to 131 classes.


In [7]:
base_model = VGG16(weights='imagenet', include_top = False , input_shape=(224,224, 3))
print(base_model.summary())

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [48]:
flatten= Flatten()(base_model.layers[-1].output)
# add new classifier layers
class_dense = Dense(1024, activation='relu')(flatten)
output = Dense(CLASSES, activation='softmax')(class_dense)
# define new model
## maybe add dropout and maybe one more dense
model_pretrained = Model(inputs=base_model.inputs, outputs=output)
# summarize
model_pretrained.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [8]:
##freeze all  base_model layers and train the last ones
for layer in base_model.layers:
    layer.trainable = False

In [50]:
model_pretrained.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [13]:
EPOCHS = 50
BATCH_SIZE = 32
STEPS_PER_EPOCH = 32
VALIDATION_STEPS = 64
MODEL_FILE = 'models/vgg16-pretrained-rmsprop.model'

In [52]:
model_pretrained.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [11]:
checkpoint = ModelCheckpoint("vgg16_1_pretrained.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0.001, patience=20, verbose=1, mode='auto')
#csv_logger = CSVLogger('./results/vgg16imagenetpretrained.csv')

In [42]:
# checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
history_pretrained = model_pretrained.fit(
    traindata,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=testdata,
    validation_steps=VALIDATION_STEPS,
    shuffle=True,
    callbacks=[checkpoint, early_stopper]
)


model.save(MODEL_FILE)

Epoch 1/50
32/32 [==============================] - 417s 13s/step - loss: 168.1206 - accuracy: 0.1816 - val_loss: 53.8305 - val_accuracy: 0.3086
Epoch 2/50
32/32 [==============================] - 434s 14s/step - loss: 21.5831 - accuracy: 0.5557 - val_loss: 23.7145 - val_accuracy: 0.4614
Epoch 3/50
32/32 [==============================] - 447s 14s/step - loss: 10.8078 - accuracy: 0.6670 - val_loss: 13.0831 - val_accuracy: 0.5815
Epoch 4/50
32/32 [==============================] - 458s 14s/step - loss: 5.7536 - accuracy: 0.7598 - val_loss: 11.7591 - val_accuracy: 0.5571
Epoch 5/50
32/32 [==============================] - ETA: 0s - loss: 5.3090 - accuracy: 0.7969

KeyboardInterrupt: 

1.2 SVM

In [54]:
print(base_model.summary())

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [14]:
class_dense_svm = Dense(1024, activation='relu')(base_model.layers[-1].output)
output_svm = Dense(1024, activation='relu')(class_dense_svm)
###own layer? different way?
# clf = make_pipeline(output_svm, SVC(gamma='auto'))

# # define new model
# ## maybe add dropout and maybe one more dense
# model_pretrained = Model(inputs=base_model.inputs, outputs=clf)
# model_pretrained.compile(optimizer='rmsprop',
#               loss='categorical_crossentropy',
#               metrics=['acc'])
# # summarize
# history_svm = model_pretrained.fit({
#     traindata,
#     epochs=EPOCHS,
#     steps_per_epoch=STEPS_PER_EPOCH,
#     validation_data=testdata,
#     validation_steps=VALIDATION_STEPS,
#     shuffle=True,
#     callbacks=[checkpoint, early_stopper]
# }
# )
# model_pretrained.summary()

SyntaxError: invalid syntax (<ipython-input-14-90591ffe53f4>, line 14)